# Deep Transfer Learning for Text Classification

Handling tough real-world problems in Natural Language Processing (NLP) include tackling with class imbalance and the lack of availability of enough labeled data for training. Thanks to the recent advancements in deep transfer learning in NLP, we have been able to make rapid strides in not only tackling these problems but also leverage these models for diverse downstream NLP tasks.

In this exercise you will be writing code to build a Hate Speech Classifier using:

- BERT (fine-tuning)
- DistilBERT (fine-tuning)

# GPU Check

In [2]:
!nvidia-smi

Mon Dec 13 18:44:59 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P8    29W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Load Necessary Dependencies

In [3]:
!pip install transformers

     |████████████████████████████████| 3.3 MB 5.4 MB/s 
     |████████████████████████████████| 895 kB 35.2 MB/s 
     |████████████████████████████████| 61 kB 496 kB/s 
     |████████████████████████████████| 596 kB 48.1 MB/s 
     |████████████████████████████████| 3.3 MB 22.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [4]:
import transformers
transformers.__version__

'4.13.0'

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_hub as hub
import transformers
import tqdm
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

%matplotlib inline

# fix random seed for reproducibility
seed = 42
np.random.seed(seed)
tf.random.set_seed(seed)

In [6]:
print("TF Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("TF Hub version: ", hub.__version__)
print("GPU is", "available" if tf.test.is_gpu_available() else "NOT AVAILABLE")

TF Version:  2.7.0
Eager mode:  True
TF Hub version:  0.12.0
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
GPU is available


## Load Dataset - Hate Speech

Social media unfortunately is rampant with hate speech in the form of posts and comments. This is a practical example of perhaps building an automated hate speech detection system using NLP in the form of text classification.

In this notebook, we will leverage an open sourced collection of hate speech posts and comments.

The dataset is available here: [kaggle](https://www.kaggle.com/usharengaraju/dynamically-generated-hate-speech-dataset) which in turn has been curated from a wider [data source for hate speech](https://hatespeechdata.com/)

In [20]:
df = pd.read_csv('HateDataset.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40623 entries, 0 to 40622
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Unnamed: 0      40623 non-null  int64  
 1   id              40623 non-null  object 
 2   text            40623 non-null  object 
 3   label           40623 non-null  object 
 4   type            40623 non-null  object 
 5   model_wrong     26097 non-null  object 
 6   db.model_preds  26097 non-null  float64
 7   status          40623 non-null  object 
 8   round           40623 non-null  object 
 9   split           40623 non-null  object 
 10  annotator       40623 non-null  object 
dtypes: float64(1), int64(1), object(9)
memory usage: 3.4+ MB


# Subset Dataset

BERT is a HUGE model which takes a long time to fine-tune!  

So let's try to subset our data so you can work with a small dataset and train faster

In [21]:
df = df[['text', 'label']]
df = df.sample(10000, random_state=42)
df.head()

,text,label
18286,I love the support of ma niggers,nothate
26266,Report all the rapists and abusers asap.,nothate
23080,No one loves fucking spicy asian food,nothate
29588,"Dear White Liberals, Malcolm X informed us abo...",nothate
6014,I believe all hefemales are the same: unbearable!,hate


# Preparing Train, Validation and Test Datasets


### Prepare Train-Test Split. Keep test set to be 20% of the total

In [22]:
from sklearn.model_selection import train_test_split

train_reviews, test_reviews, train_labels, test_labels = train_test_split(df.text.values,
                                                                          df.label.values,
                                                                          test_size=0.2, random_state=42)

In [23]:
len(train_reviews), len(test_reviews)

(8000, 2000)

# No Text Pre-processing

__Note:__ For some models we don't use any pre-processing like BERT! It should be able to handle a wide variety of text in its natural format

## Label Encode Classes

# **Question 1**: Label Encode Class Labels

`hate` and `nothate` needs to be encoded to numbers

In [24]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

y_train = le.fit_transform(train_labels) # fit - transform on train labels
y_test = le.transform(test_labels)  # transform on test labels

# Model 1: BERT (Bi-directional Encoder Representations from Transformers)

We will be using the BERT base model which has already been pre-trained by Google on MLM + Next Sentence Prediction Tasks

## BERT Tokenization

The BERT model we're using expects lowercase data. Here we leverage Huggingface `transformers`' `BertTokenizer`, which breaks words into word pieces.

Word Piece Tokenizer is based on [Byte Pair Encodings (BPE)](https://www.aclweb.org/anthology/P16-1162).

WordPiece and BPE are two similar and commonly used techniques to segment words into subword-level in NLP tasks. In both cases, the vocabulary is initialized with all the individual characters in the language, and then the most frequent/likely combinations of the symbols in the vocabulary are iteratively added to the vocabulary.

magine that the model sees the word walking. Unless this word occurs at least a few times in the training corpus, the model can't learn to deal with this word very well. However, it may have the words walked, walker, walks, each occurring only a few times. Without subword segmentation, all these words are treated as completely different words by the model.

However, if these get segmented as walk@@ ing, walk@@ ed, etc., notice that all of them will now have walk@@ in common, which will occur much frequently while training, and the model might be able to learn more about it.

Huggingface's transformers library has easy to use utilities for each type of model

# **Question 2**: Load BERT Tokenizer

_Hint: Use ``transformers.BertTokenizer.from_pretrained`` with the right pretrained model (bert base uncased) similar to the tutorial_

In [25]:
tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-uncased')

## BERT Data Preparation

We need to preprocess our data so that it matches the data format BERT was trained on. For this, we'll need to do a couple of things.

- Lowercase our text (if we're using a BERT lowercase model)
- Tokenize it (i.e. "sally says hi" -> ["sally", "says", "hi"])
- Break words into WordPieces (i.e. "calling" -> ["call", "##ing"])
- Map our words to indexes using a vocab file that BERT provides
- Add special "CLS" and "SEP" tokens (see the readme)
- Append "mask" and "segment" tokens to each input (see the BERT paper)

# **Question 3**: Create function to tokenize and encode text into BERT token IDs

Use a similar strategy as you learnt in the tutorial and fill in the following function

In [40]:
def create_bert_input_features(tokenizer, docs, max_seq_length):
    
    all_ids, all_masks, all_segments= [], [], []
    for doc in tqdm.tqdm(docs, desc="Converting docs to features"):
        
        tokens = tokenizer.tokenize(doc)
        
        if len(tokens) > max_seq_length-2:
            tokens = tokens[0 : (max_seq_length-2)]
        tokens = ['[CLS]'] + tokens + ['[SEP]']
        ids = tokenizer.convert_tokens_to_ids(tokens)
        masks = [1] * len(ids) # [1,1,1.....]

        while len(ids) < max_seq_length:
            ids.append(0)
            masks.append(0)
            
        segments = [0] * max_seq_length # [0,0,0...]
        all_ids.append(ids)
        all_masks.append(masks)
        all_segments.append(segments)
        
    encoded = np.array([all_ids, all_masks, all_segments])
    
    return encoded


# **Question 4**: Build Model Architecture

Use a similar model architecture as the tutorial with a max sequence length of 500 tokens to be used.

You can always experiment with aspects like learning rate, number of layers etc.

In [41]:
MAX_SEQ_LENGTH = 500

inp_id = tf.keras.layers.Input(shape=(MAX_SEQ_LENGTH,), dtype='int32', name="bert_input_ids")
inp_mask = tf.keras.layers.Input(shape=(MAX_SEQ_LENGTH,), dtype='int32', name="bert_input_masks")
inp_segment = tf.keras.layers.Input(shape=(MAX_SEQ_LENGTH,), dtype='int32', name="bert_segment_ids")
inputs = [inp_id, inp_mask, inp_segment]

hidden_state = transformers.TFBertModel.from_pretrained('bert-base-uncased')(inputs)
pooled_output = hidden_state[1]

dense1 = tf.keras.layers.Dense(256, activation='relu')(pooled_output)
drop1 = tf.keras.layers.Dropout(0.25)(dense1)
dense2 = tf.keras.layers.Dense(256, activation='relu')(drop1)
drop2 = tf.keras.layers.Dropout(0.25)(dense2)

output = tf.keras.layers.Dense(1, activation='sigmoid')(drop2)

model = tf.keras.Model(inputs=inputs, outputs=output)

model.compile(optimizer=tf.optimizers.Adam(learning_rate=2e-5, 
                                           epsilon=1e-08), 
              loss='binary_crossentropy', metrics=['accuracy'])

model.summary()

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 bert_input_ids (InputLayer)    [(None, 500)]        0           []                               
                                                                                                  
 bert_input_masks (InputLayer)  [(None, 500)]        0           []                               
                                                                                                  
 bert_segment_ids (InputLayer)  [(None, 500)]        0           []                               
                                                                                                  
 tf_bert_model_5 (TFBertModel)  TFBaseModelOutputWi  109482240   ['bert_input_ids[0][0]',         
                                thPoolingAndCrossAt               'bert_input_masks[0][0]', 

## Convert text to BERT input features

We leverage our utility function we created earlier to convert our text documents into BERT input features.

# **Question 5**: Prepare Feature ID, masks and segments for train and test sets

In [42]:
train_features_ids, train_features_masks, train_features_segments = create_bert_input_features(tokenizer, 
                                                                                               train_reviews, 
                                                                                               max_seq_length=MAX_SEQ_LENGTH)
test_features_ids, test_features_masks, test_features_segments = create_bert_input_features(tokenizer, 
                                                                                               test_reviews, 
                                                                                               max_seq_length=MAX_SEQ_LENGTH)

print('Train Features:', train_features_ids.shape, train_features_masks.shape, train_features_segments.shape)
print('Test Features:', test_features_ids.shape, test_features_masks.shape, test_features_segments.shape)

Converting docs to features: 100%|██████████| 2000/2000 [00:01<00:00, 1217.95it/s]


Train Features: (8000, 500) (8000, 500) (8000, 500)
Test Features: (2000, 500) (2000, 500) (2000, 500)


## Train and Validate Model

# **Question 6**: Train the Model

You can train it for around 3 epochs as it takes a fair bit of time to train even on a good GPU

For validation data you can use the test data tokens from the previous cell to keep things simple

In [48]:
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                      patience=1,
                                      restore_best_weights=True,
                                      verbose=1)
model.fit([train_features_ids, 
           train_features_masks, 
           train_features_segments], train_labels, 
          validation_data=([test_features_ids, 
                            test_features_masks, 
                            test_features_segments], test_labels),
          epochs=3, 
          batch_size=13, 
          callbacks=[es],
          shuffle=True,
          verbose=1)

Epoch 1/3


UnimplementedError: ignored

# **Question 7**: Model Performance Evaluation on the Test Dataset

Show the model's performance on the test dataset

In [49]:
predictions = [1 if pr > 0.5 else 0 
                   for pr in model.predict([test_features_ids, 
                                            test_features_masks, 
                                            test_features_segments], verbose=0).ravel()]

print("Accuracy: %.2f%%" % (accuracy_score(test_labels, predictions)*100))
print(classification_report(test_labels, predictions))
pd.DataFrame(confusion_matrix(test_labels, predictions))

TypeError: ignored

# Model 2: DistilBERT (Distilled BERT)


## BERT Tokenization

The DistilBERT model we're using expects lowercase data. Here we leverage Huggingface `transformers`' `DistilBertTokenizer`, which breaks words into word pieces.

Word Piece Tokenizer is based on [Byte Pair Encodings (BPE)](https://www.aclweb.org/anthology/P16-1162).

WordPiece and BPE are two similar and commonly used techniques to segment words into subword-level in NLP tasks. In both cases, the vocabulary is initialized with all the individual characters in the language, and then the most frequent/likely combinations of the symbols in the vocabulary are iteratively added to the vocabulary.

magine that the model sees the word walking. Unless this word occurs at least a few times in the training corpus, the model can't learn to deal with this word very well. However, it may have the words walked, walker, walks, each occurring only a few times. Without subword segmentation, all these words are treated as completely different words by the model.

However, if these get segmented as walk@@ ing, walk@@ ed, etc., notice that all of them will now have walk@@ in common, which will occur much frequently while training, and the model might be able to learn more about it.

Huggingface's transformers library has easy to use utilities for each type of model

# **Question 8**: Load DistilBERT Tokenizer

_Hint: Use ``transformers.DistilBertTokenizer.from_pretrained`` with the right pretrained model (distilbert base uncased) similar to the tutorial_

In [ ]:
tokenizer = transformers.DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

## DistilBERT Data Preparation

We need to preprocess our data so that it matches the data format DistilBERT was trained on. For this, we'll need to do a couple of things.

- Lowercase our text (if we're using a BERT lowercase model)
- Tokenize it (i.e. "sally says hi" -> ["sally", "says", "hi"])
- Break words into WordPieces (i.e. "calling" -> ["call", "##ing"])
- Map our words to indexes using a vocab file that BERT provides
- Add special "CLS" and "SEP" tokens (see the readme)
- Append "mask" tokens to each input (see https://medium.com/huggingface/distilbert-8cf3380435b5)

# **Question 9**: Create function to tokenize and encode text into DistilBERT token IDs

Use a similar strategy as you learnt in the tutorial and fill in the following function

In [ ]:
def create_bert_input_features(tokenizer, docs, max_seq_length):
    
    all_ids, all_masks = [], []
    for doc in tqdm.tqdm(docs, desc="Converting docs to features"):
        
        tokens = tokenizer.tokenize(doc)
        
        if len(tokens) > max_seq_length-2:
            tokens = tokens[0 : (max_seq_length-2)]
        tokens = ['[CLS]'] + tokens + ['[SEP]']
        ids = tokenizer.convert_tokens_to_ids(tokens)
        masks = [1] * len(ids)
        
        while len(ids) < max_seq_length:
            ids.append(0)
            masks.append(0)
            
        all_ids.append(ids)
        all_masks.append(masks)
        
    encoded = np.array([all_ids, all_masks])
    
    return encoded

# **Question 10**: Build Model Architecture

Use a similar model architecture as the tutorial with a max sequence length of 500 tokens to be used.

You can always experiment with aspects like learning rate, number of layers etc.

In [ ]:
MAX_SEQ_LENGTH = 500

inp_id = tf.keras.layers.Input(shape=(MAX_SEQ_LENGTH,), dtype='int32', name="bert_input_ids")
inp_mask = tf.keras.layers.Input(shape=(MAX_SEQ_LENGTH,), dtype='int32', name="bert_input_masks")
inputs = [inp_id, inp_mask]

hidden_state = transformers.TFDistilBertModel.from_pretrained('distilbert-base-uncased')(inputs)[0]
pooled_output = hidden_state[:, 0]    

dense1 = tf.keras.layers.Dense(256, activation='relu')(pooled_output)
drop1 = tf.keras.layers.Dropout(0.25)(dense1)
dense2 = tf.keras.layers.Dense(256, activation='relu')(drop1)
drop2 = tf.keras.layers.Dropout(0.25)(dense2)

output = tf.keras.layers.Dense(1, activation='sigmoid')(drop2)


model = tf.keras.Model(inputs=inputs, outputs=output)
model.compile(optimizer=tf.optimizers.Adam(learning_rate=2e-5, 
                                           epsilon=1e-08), 
              loss='binary_crossentropy', metrics=['accuracy'])

model.summary()

# **Question 11**: Prepare Feature ID, masks and segments for train and test sets

In [ ]:
train_features_ids, train_features_masks = create_bert_input_features(tokenizer, train_reviews, 
                                                                      max_seq_length=MAX_SEQ_LENGTH)
test_features_ids, test_features_masks = create_bert_input_features(tokenizer, test_reviews, 
                                                                      max_seq_length=MAX_SEQ_LENGTH)

print('Train Features:', train_features_ids.shape, train_features_masks.shape)
print('Val Features:', test_features_ids.shape, test_features_masks.shape)

## Train and Validate Model

# **Question 12**: Train the Model

You can train it for around 3 epochs as it takes a fair bit of time to train even on a good GPU

For validation data you can use the test data tokens from the previous cell to keep things simple

In [ ]:
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                      patience=1,
                                      restore_best_weights=True,
                                      verbose=1)
model.fit([train_features_ids, 
           train_features_masks], train_labels, 
          validation_data=([test_features_ids, 
                            test_features_masks], test_labels),
          epochs=3, 
          batch_size=20, 
          shuffle=True,
          callbacks=[es],
          verbose=1)

# **Question 13**: Model Performance Evaluation on the Test Dataset

Show the model's performance on the test dataset

In [ ]:
predictions = [1 if pr > 0.5 else 0 
                   for pr in model.predict([test_features_ids, 
                                            test_features_masks], batch_size=200, verbose=0).ravel()]

print("Accuracy: %.2f%%" % (accuracy_score(test_labels, predictions)*100))
print(classification_report(test_labels, predictions))
pd.DataFrame(confusion_matrix(test_labels, predictions))